In [1]:
from collections import defaultdict
import numpy as np 
import pandas as pd 
import os
from util_get_data import *
provinces = ['Khon Kaen','Chiang Mai','Chanthaburi','Bangkok','Kanchanaburi','Songkhla']
base_url = '../'
dest_url = './extracted/'

pd.__version__

'1.2.4'

## Raw -> tables

In [2]:
target = defaultdict(lambda: {})
wind = defaultdict(lambda: {})
temp = defaultdict(lambda: {})

for subset in ['Train', 'Test']:
    for p in provinces:
        target_df = get_target_df(province=p, subset=subset)
        wind_df = get_df(measure='wind', province=p, subset=subset)
        temp_df = get_df(measure='temperature', province=p, subset=subset)

        target[subset][p] = target_df.resample('H').mean()
        wind[subset][p] = wind_df
        temp[subset][p] = temp_df

        print(f'{subset} : {p} : {target[subset][p].shape}')

Train : Khon Kaen : (26632, 1)
Train : Chiang Mai : (26632, 1)
Train : Chanthaburi : (26632, 1)
Train : Bangkok : (26632, 1)
Train : Kanchanaburi : (26632, 1)
Train : Songkhla : (26632, 1)
Test : Khon Kaen : (8797, 1)
Test : Chiang Mai : (8797, 1)
Test : Chanthaburi : (8797, 1)
Test : Bangkok : (8797, 1)
Test : Kanchanaburi : (8797, 1)
Test : Songkhla : (8797, 1)


## Save raw extracted files

In [ ]:
'''
for subset in ['Train', 'Test']:
    for p in provinces:
        target[subset][p].to_csv(f'./extracted/{subset}/{p}_target.csv')
        wind[subset][p].to_csv(f'./extracted/{subset}/{p}_wind.csv')
        temp[subset][p].to_csv(f'./extracted/{subset}/{p}_temp.csv')
'''

## Join PM, wind, temperature

In [3]:
full_all = defaultdict(lambda: {})
outer_join = dict(left_index=True, right_index=True, how='outer')
left_join = dict(how='left', left_index=True, right_index=True)

for subset in ['Train','Test']:
    for p in provinces:
        # Full timeline 
        idx = pd.date_range(target[subset][p].index[0], 
                        target[subset][p].index[-1], freq='H')
        timeline = pd.DataFrame(index=idx)

        target_full = timeline.merge(target[subset][p], **left_join)

        # Join wind &temperature
        wind_df = wind[subset][p].drop(['lat','long'],axis=1)
        temp_df = temp[subset][p].drop(['lat','long'],axis=1)

        wind_temp = pd.merge(timeline, 
                        wind_df.merge(temp_df, **outer_join), 
                        **left_join).ffill(limit=3)

        
        full = target_full.merge(wind_temp, **left_join)

        ## left = left.resample('H').mean().ffill(limit=3)

        # Join PM, wind, temperature
        #full = wind_temp_full.merge(target[subset][p], **left_join)

        full_all[subset][p] = full
        print(f"{subset} : {p} : {full.shape}")

Train : Khon Kaen : (26632, 4)
Train : Chiang Mai : (26632, 4)
Train : Chanthaburi : (26632, 4)
Train : Bangkok : (26632, 4)
Train : Kanchanaburi : (26632, 4)
Train : Songkhla : (26632, 4)
Test : Khon Kaen : (8797, 4)
Test : Chiang Mai : (8797, 4)
Test : Chanthaburi : (8797, 4)
Test : Bangkok : (8797, 4)
Test : Kanchanaburi : (8797, 4)
Test : Songkhla : (8797, 4)


## Save joined 3 to full-data

In [4]:
for subset in ['Train', 'Test']:
    for p in provinces:
        path = f'./full-data/{subset}/{p}_full.csv'
        if not os.path.exists(path): full_all[subset][p].to_csv(path)
        else: print(path)

./full-data/Train/Khon Kaen_full.csv
./full-data/Train/Chiang Mai_full.csv
./full-data/Train/Chanthaburi_full.csv
./full-data/Train/Bangkok_full.csv
./full-data/Train/Kanchanaburi_full.csv
./full-data/Train/Songkhla_full.csv
./full-data/Test/Khon Kaen_full.csv
./full-data/Test/Chiang Mai_full.csv
./full-data/Test/Chanthaburi_full.csv
./full-data/Test/Bangkok_full.csv
./full-data/Test/Kanchanaburi_full.csv
./full-data/Test/Songkhla_full.csv


# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [5]:
raise Exception("Don't run code below. it's deprecated")

Exception: Don't run code below. it's deprecated

In [2]:
def get_target_df(target_url, subset):

    if subset == 'Test':
        return pd.read_csv(target_url, index_col=0)

    with open(target_url, 'r') as f:
        for i, line in enumerate(f):
            if i==9:
                cols = line.strip().split(', ')
                break
            
    target_df = pd.read_csv(target_url, sep='\t', skiprows=10, header=None)
    target_df.columns = cols
    target_df.iloc[:,:4] = target_df.iloc[:,:4].applymap(str)
    target_df['Date'] = pd.to_datetime(
                                       {'year': target_df['% Year'], 
                                        'month': target_df['Month'], 
                                        'day':target_df['Day'],
                                        'hour': target_df['UTC Hour']})
    target_df = target_df[['Date','PM2.5']].set_index('Date')
    return target_df.groupby('Date').mean()

def get_df(url):
    return pd.read_csv(url, parse_dates=['datetime'], index_col='datetime')

In [4]:


for subset in ['Train', 'Test']:
    if len(os.listdir(dest_url + subset)) == 0:
        for p in provinces:
            if p in ['Khon Kaen','Chiang Mai']:
                p_ = p.replace(' ','_')
                target_url = base_url+'{}/{}/{}.'.format(p,subset,p_)
            else:
                target_url = base_url+'{}/{}/{}.'.format(p,subset,p)
            if subset == 'Test':
                target_url+='csv'
            elif subset == 'Train':
                target_url+='txt'

            wind_url = base_url+'{}/{}/3H_wind_{}.csv'.format(p,subset,p)
            temp_url = base_url+'{}/{}/3H_temperature_{}.csv'.format(p,subset,p)
            
            get_target_df(target_url, subset).to_csv(dest_url+f'{subset}/'+'{}_target.csv'.format(p))
            get_df(wind_url).to_csv(dest_url+f'{subset}/'+'{}_wind.csv'.format(p))
            get_df(temp_url).to_csv(dest_url+f'{subset}/'+'{}_temp.csv'.format(p))
            print('{}/{} done!!'.format(subset,p))
    else:
        print(f'{subset} Preprocessed!')

Train Preprocessed!
Test Preprocessed!


## Read TRAIN tables

In [7]:
temp = dict()
wind = dict()
target = dict()

subset = 'Test'

for p in provinces:
    target[p] = pd.read_csv(dest_url+'/{}/{}_target.csv'.format(subset,p), index_col=0, parse_dates=True)
    wind[p] = pd.read_csv(dest_url+'/{}/{}_wind.csv'.format(subset,p), parse_dates=['datetime'], index_col='datetime')
    temp[p] = pd.read_csv(dest_url+'/{}/{}_temp.csv'.format(subset,p), parse_dates=['datetime'], index_col='datetime')

## Joining tables 

In [8]:
print(f'subset : {subset}')
for p in provinces:
    print("temp : {} : shape".format(p),temp[p].shape)
    print("wind : {} : shape".format(p),wind[p].shape)

subset : Test
temp : Khon Kaen : shape (2936, 3)
wind : Khon Kaen : shape (2936, 4)
temp : Chiang Mai : shape (2936, 3)
wind : Chiang Mai : shape (2936, 4)
temp : Chanthaburi : shape (2936, 3)
wind : Chanthaburi : shape (2936, 4)
temp : Bangkok : shape (2936, 3)
wind : Bangkok : shape (2936, 4)
temp : Kanchanaburi : shape (2936, 3)
wind : Kanchanaburi : shape (2936, 4)
temp : Songkhla : shape (2936, 3)
wind : Songkhla : shape (2935, 4)


ข้อมูล Wind กับ Temp เป็นข้อมูลที่มี freq = 3H ซึ่งเราต้องเอาไป join กับ target ที่มี freq = H

In [9]:
for province in provinces:
    # ดู date_rage ของข้อมูล PM2.5 แต่ละจังหวัด
    target_timeline = pd.date_range(target[province].index[0], target[province].index[-1], freq='H')

    print(f'{province} : full timeline : {target_timeline.shape}')

Khon Kaen : full timeline : (8797,)
Chiang Mai : full timeline : (8797,)
Chanthaburi : full timeline : (8797,)
Bangkok : full timeline : (8797,)
Kanchanaburi : full timeline : (8797,)
Songkhla : full timeline : (8797,)


แปลว่าทุกๆ Train data เริ่มและจบที่จุดเดียวกัน -> ใช้ `target_timeline` เป็นไทม์ไลน์หลัก

In [10]:
target['Bangkok'].shape

(8797, 1)

In [11]:
# Join : temperature | wind
temp_wind_joined = dict()
lat_long = dict()

for p in provinces:
    lat_long[p] = {'lat': temp[p]['lat'].unique()[0], 'long': temp[p]['long'].unique()[0]}
    wind[p] = wind[p].drop(['lat','long'], axis=1)
    temp[p] = temp[p].drop(['lat', 'long'], axis=1)

    temp_wind_joined[p] = temp[p].merge(
                            wind[p],
                            how='left',
                            left_index=True,
                            right_index=True
                        )

In [12]:
full_data = dict()
'''
for province in provinces:
    f = target[province].copy()
    f = f.append(pd.DataFrame(index=pd.DatetimeIndex(data=['2016-03-03 07:00:00'])))
    f = f.sort_index()
    f = f.resample(rule='3H', closed='left', origin='start').mean()

    full_data[province] = temp_wind_joined[province].merge(f, left_index=True, right_index=True)
    full_data[province].index.name = 'Datetime'
'''
# Global timeline
timeline = pd.DataFrame(index=target_timeline)
for province in provinces:
    # Timeline for each province
    timeline = pd.DataFrame(index=pd.date_range(target[province].index[0], 
                                                target[province].index[0], freq='H'))
    full_data[province] = timeline.merge(target[province],
                                left_index=True, 
                                right_index=True,
                                how='left')
    print(f"target : {province} : {full_data[province].shape}")

target : Khon Kaen : (8797, 1)
target : Chiang Mai : (8797, 1)
target : Chanthaburi : (8797, 1)
target : Bangkok : (8797, 1)
target : Kanchanaburi : (8797, 1)
target : Songkhla : (8797, 1)


เตรียม target เสร็จเรียบร้อย ต่อไปจะทำการ join (temp|wind ที่ถูก resample เป็น 1H และ fill missing by `ffill`)เข้ากับ target

In [13]:
for province in provinces:

    df = temp_wind_joined[province].resample('H').mean().fillna(method='ffill')
    full_data[province] = full_data[province].merge(df, right_index=True, left_index=True, how='left')

In [17]:
print(f'From total records : {target_timeline.shape}')
print(full_data['Bangkok'].columns.values, end='\n\n')
for province in provinces:
    print(f'{province} : Null value : {full_data[province].isnull().sum().values}')
    if subset == 'Test':
        full_data[province].rename(columns={"PM2.5(µg/m3)":"PM2.5"}, inplace=True)

From total records : (8797,)
['PM2.5(µg/m3)' 'Temp(C)' 'WindDir' 'Wind Speed(km/h)']

Khon Kaen : Null value : [0 0 0 0]
Chiang Mai : Null value : [0 0 0 0]
Chanthaburi : Null value : [0 0 0 0]
Bangkok : Null value : [0 0 0 0]
Kanchanaburi : Null value : [0 0 0 0]
Songkhla : Null value : [0 0 0 0]


In [18]:
print(f'subset : {subset}')
if len(os.listdir('./full-data/' + subset)) == 0:
    for province in provinces:
        full_data[province].to_csv(f'./full-data/{subset}/'+province+'_full.csv')
else:
    print('already')

subset : Test
